# Material Symbols SVG → PNG (CairoSVG) — Multi-Color + Multi-Size

This notebook converts a folder (or ZIP) of **Material Symbols SVG** icons into **PNG** files in multiple **brand colors** and **sizes**, organized into per-color folders (and optional size subfolders).

✅ Best renderer for Google Material Symbols SVGs (handles modern SVG/CSS correctly)
✅ Outputs crisp PNGs for **Power BI** (recommended size: **48px**)

---
## You will edit only the **Configuration** cell.


In [9]:
# Install (run once). If you're using Anaconda, you may prefer the conda-forge route:
# conda install -c conda-forge cairosvg cairo cairocffi
%pip install -q cairosvg pillow tqdm

Note: you may need to restart the kernel to use updated packages.


## 1) Configuration
Set either:
- `INPUT_SVG_DIR` (a folder containing SVGs), **OR**
- `INPUT_ZIP_PATH` (a ZIP containing SVGs)

Set output + colors + sizes.


In [10]:
from pathlib import Path

# === Choose ONE input method ===
INPUT_SVG_DIR  = Path(r"C:\Users\Elphys\OneDrive\Desktop\IconsOrange")   # folder with SVGs
INPUT_ZIP_PATH = None  # e.g. Path(r"C:\Users\Elphys\Desktop\IconsOrange.zip")

# Output folder
OUTPUT_DIR = Path(r"C:\Users\Elphys\OneDrive\Desktop\IconsPNG_Export")

# Brand colors (add more anytime)
COLORS = [
    "#F05A28",
    "#102040",
    "#204050",
    "#203040",
    "#E04A1C",
    "#E0E0E0",
    "#F0F0F0",
]

# Sizes in pixels (Power BI recommendation: 48)
SIZES = [48]     # or [24, 48]

# Folder layout
NEST_BY_SIZE = True  # True => COLOR/size_48/icon.png ; False => COLOR/icon_48.png

# Transparent background (recommended)
TRANSPARENT_BG = True

# --- sanity checks ---
if INPUT_ZIP_PATH is None:
    assert INPUT_SVG_DIR.exists(), f"INPUT_SVG_DIR not found: {INPUT_SVG_DIR}"
else:
    assert INPUT_ZIP_PATH.exists(), f"INPUT_ZIP_PATH not found: {INPUT_ZIP_PATH}"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("OUTPUT_DIR:", OUTPUT_DIR)
print("COLORS:", COLORS)
print("SIZES:", SIZES)
print("NEST_BY_SIZE:", NEST_BY_SIZE)


OUTPUT_DIR: C:\Users\Elphys\OneDrive\Desktop\IconsPNG_Export
COLORS: ['#F05A28', '#102040', '#204050', '#203040', '#E04A1C', '#E0E0E0', '#F0F0F0']
SIZES: [48]
NEST_BY_SIZE: True


## 2) Helpers (recolor + naming)
We recolor by:
1) replacing `currentColor` with your target hex
2) replacing the **most common hex color** in the SVG (usually the original icon color) with your target hex

This works well for Material Symbols downloads like `..._F05A28_...svg`.


In [11]:
import re
from collections import Counter

HEX_RE = re.compile(r"#[0-9a-fA-F]{6}")

def sanitize_hex(h: str) -> str:
    h = h.strip()
    if not h.startswith("#"):
        h = "#" + h
    if len(h) != 7 or not HEX_RE.fullmatch(h):
        raise ValueError(f"Invalid hex color: {h}")
    return h.upper()

def most_common_hex(svg_text: str) -> str | None:
    matches = [m.upper() for m in HEX_RE.findall(svg_text)]
    if not matches:
        return None
    return Counter(matches).most_common(1)[0][0]

def recolor_svg(svg_text: str, target_hex: str) -> str:
    target_hex = sanitize_hex(target_hex)

    # 1) Replace currentColor (common in Material Symbols)
    out = re.sub(r"currentColor", target_hex, svg_text, flags=re.IGNORECASE)

    # 2) Replace the most common explicit hex (often the original fill color)
    common = most_common_hex(out)
    if common and common.upper() != target_hex.upper():
        out = re.sub(re.escape(common), target_hex, out, flags=re.IGNORECASE)

    return out

def safe_icon_basename(svg_path: Path) -> str:
    # Examples:
    # analytics_24dp_F05A28_FILL0_wght400_GRAD0_opsz24.svg
    # engineering_24dp_F05A28_FILL0_wght400_GRAD0_opsz24 (1).svg
    stem = svg_path.stem
    stem = re.sub(r"\s*\(\d+\)$", "", stem)  # drop trailing " (1)"
    m = re.match(r"^(.*?)(_[0-9]+dp).*?$", stem)
    return m.group(1) if m else stem

def color_folder_name(hex_color: str) -> str:
    return sanitize_hex(hex_color).replace("#", "")


## 3) CairoSVG export
If CairoSVG imports successfully, rendering should work.


In [12]:
import cairosvg

print("cairosvg version:", getattr(cairosvg, "__version__", "unknown"))

def export_png(svg_text: str, out_path: Path, size_px: int, transparent_bg: bool = True) -> None:
    out_path.parent.mkdir(parents=True, exist_ok=True)
    bg = "transparent" if transparent_bg else None

    cairosvg.svg2png(
        bytestring=svg_text.encode("utf-8"),
        write_to=str(out_path),
        output_width=int(size_px),
        output_height=int(size_px),
        background_color=bg,
    )


cairosvg version: 2.8.2


## 4) Load SVGs (from folder or ZIP)


In [13]:
import zipfile
import tempfile

def get_svg_paths():
    if INPUT_ZIP_PATH is None:
        return sorted(INPUT_SVG_DIR.glob("*.svg")), None

    tmpdir = Path(tempfile.mkdtemp(prefix="icons_svg_"))
    with zipfile.ZipFile(INPUT_ZIP_PATH, "r") as z:
        z.extractall(tmpdir)
    svg_paths = sorted(tmpdir.rglob("*.svg"))
    return svg_paths, tmpdir

svg_paths, extracted_tmpdir = get_svg_paths()
print("SVG count:", len(svg_paths))
print("Example:", svg_paths[:3])
assert len(svg_paths) > 0, "No SVG files found. Check your INPUT path."


SVG count: 86
Example: [WindowsPath('C:/Users/Elphys/OneDrive/Desktop/IconsOrange/add_call_24dp_F05A28_FILL0_wght400_GRAD0_opsz24 (1).svg'), WindowsPath('C:/Users/Elphys/OneDrive/Desktop/IconsOrange/add_call_24dp_F05A28_FILL0_wght400_GRAD0_opsz24.svg'), WindowsPath('C:/Users/Elphys/OneDrive/Desktop/IconsOrange/add_home_work_24dp_F05A28_FILL0_wght400_GRAD0_opsz24.svg')]


## 5) Run batch export
Outputs:
- `OUTPUT_DIR/<COLOR>/size_<N>/<icon>.png` if `NEST_BY_SIZE=True`
- `OUTPUT_DIR/<COLOR>/<icon>_<N>.png` if `NEST_BY_SIZE=False`


In [14]:
from tqdm.auto import tqdm

errors = []
written = 0

total = len(svg_paths) * len(COLORS) * len(SIZES)
print(f"Will attempt: {total} PNG renders")

for svg_path in tqdm(svg_paths, desc="SVG files"):
    try:
        raw = svg_path.read_text(encoding="utf-8", errors="ignore")
    except Exception as e:
        errors.append((svg_path.name, "read", str(e)))
        continue

    icon_name = safe_icon_basename(svg_path)

    for color in COLORS:
        c = sanitize_hex(color)
        recolored = recolor_svg(raw, c)
        color_dir = OUTPUT_DIR / color_folder_name(c)

        for size in SIZES:
            try:
                if NEST_BY_SIZE:
                    out_path = color_dir / f"size_{size}" / f"{icon_name}.png"
                else:
                    out_path = color_dir / f"{icon_name}_{size}.png"

                export_png(recolored, out_path, size_px=size, transparent_bg=TRANSPARENT_BG)
                written += 1
            except Exception as e:
                errors.append((svg_path.name, f"color={c} size={size}", str(e)))

print(f"Done. Wrote {written} PNGs.")
print(f"Errors: {len(errors)}")


Will attempt: 602 PNG renders


SVG files:   0%|          | 0/86 [00:00<?, ?it/s]

Done. Wrote 602 PNGs.
Errors: 0


## 6) If there are errors, show the first 20


In [15]:
for err in errors[:20]:
    print(err)


## 7) Optional: Zip the output folder


In [16]:
import shutil

archive_path = shutil.make_archive(str(OUTPUT_DIR), "zip", root_dir=str(OUTPUT_DIR))
print("Created zip:", archive_path)


Created zip: C:\Users\Elphys\OneDrive\Desktop\IconsPNG_Export.zip


---
# README / Notes

### Add more colors
Add to the `COLORS` list, e.g.:
```python
COLORS.append('#00A3E0')
```

### Change sizes
Edit `SIZES`, e.g.:
```python
SIZES = [24, 48, 96]
```

### Power BI recommendation
- Use **48px** for most dashboard icons.
- Power BI scales **down** better than it scales up.
